In [1]:
import numpy as np
import pandas as pd

In [2]:
def createuber():
    #this function reads in uber.csv and returns the data in pd.DataFrame format
    uber = pd.read_csv('uber.csv')
    uber.pickup_time = pd.to_datetime(uber.pickup_time)
    return uber

In [23]:
def createtaxi():
    #this function reads in yellow_trip_{year}-{month}.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        for j in range(1,13):
            if j <10:
                filename = f'yellow_tripdata_{i}-0{j}.csv'
                filenames.append(filename)
            else:
                filename = f'yellow_tripdata_{i}-{j}.csv'
                filenames.append(filename)
    taxi = pd.read_csv('yellow_tripdata_2009-01.csv',usecols = ['pickup_time','dropoff_time','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','charge','tip','distance'],nrows = 2381)
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i],usecols = ['pickup_time','dropoff_time','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','charge','tip','distance'], nrows = 2381)
        taxi = pd.concat([taxi,df])
    taxi.reset_index(inplace = True, drop = True)
    taxi.pickup_time = pd.to_datetime(taxi.pickup_time)
    taxi.dropoff_time = pd.to_datetime(taxi.dropoff_time)
    return taxi

In [4]:
def createhourlyweather():
    #this function reads in {year}_weather_hours.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        filenames.append(f'{i}_weather_hours.csv')
    weather_hourly = pd.read_csv('2009_weather_hours.csv')
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i])
        weather_hourly = pd.concat([weather_hourly,df])
    weather_hourly.reset_index(inplace = True, drop = True)
    weather_hourly.DATE = pd.to_datetime(weather_hourly.DATE)
    weather_hourly['DATE'] = weather_hourly['DATE'].dt.floor('H')
    weather_hourly = weather_hourly.drop_duplicates('DATE',keep = 'first')
    return weather_hourly
    

In [5]:
def createdailyweather():
    #this function reads in {year}_weather_days.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        filenames.append(f'{i}_weather_days.csv')
    weather_daily = pd.read_csv('2009_weather_days.csv')
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i])
        weather_daily = pd.concat([weather_daily,df])
    weather_daily.reset_index(inplace = True, drop = True)
    weather_daily.DATE = pd.to_datetime(weather_daily.DATE)
    return weather_daily

In [24]:
uber = createuber()
taxi = createtaxi()
weather_hourly= createhourlyweather()
weather_daily = createdailyweather()

In [7]:
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///ubertaxi.db", echo=True)

In [8]:
import logging
sqllogger = logging.getLogger("sqlalchemy.engine.Engine")
formatter = logging.Formatter("[sqlalchemy] %(message)s")
sqllogger.handlers[0].setFormatter(formatter)

In [9]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [10]:
from sqlalchemy import Column, Integer, String

In [11]:
Base.metadata.create_all(engine, checkfirst=True)

[sqlalchemy] BEGIN (implicit)
[sqlalchemy] COMMIT


In [16]:
from sqlalchemy.orm import sessionmaker

# sessionmaker returns a Session class
Session = sessionmaker(bind=engine)
# and we create an instance of Session
session = Session()

In [17]:
# Open the .sql file
sql_file = open('schema.sql','r')

# Create an empty command string
sql_command = ''

# Iterate over all lines in the sql file
for line in sql_file:
    # Ignore commented lines
    if not line.startswith('--') and line.strip('\n'):
        # Append line to the command string
        sql_command += line.strip('\n')

        # If the command string ends with ';', it is a full statement
        if sql_command.endswith(';'):
            # Try to execute statement and commit it
            try:
                session.execute(text(sql_command))
                session.commit()

            # Assert in case of error
            except:
                print('Ops')

            # Finally, clear command string
            finally:
                sql_command = ''

[sqlalchemy] BEGIN (implicit)
[sqlalchemy] CREATE TABLE daily_weather(date Datetime,windspeed FLOAT,precipitation FLOAT);
[sqlalchemy] [generated in 0.00082s] ()
[sqlalchemy] COMMIT
[sqlalchemy] BEGIN (implicit)
[sqlalchemy] CREATE TABLE hourly_weather(date Datetime,windspeed FLOAT,precipitation FLOAT);
[sqlalchemy] [generated in 0.00103s] ()
[sqlalchemy] COMMIT
[sqlalchemy] BEGIN (implicit)
[sqlalchemy] CREATE TABLE yellow_taxi(pickup_time DATETIME,dropoff_time DATETIME,pickup_longitude FLOAT,dropoff_longitude FLOAT,pickup_latitude FLOAT,dropoff_latitude FLOAT,tip FLOAT,charge FLOAT,distance FLOAT);
[sqlalchemy] [generated in 0.00051s] ()
[sqlalchemy] COMMIT
[sqlalchemy] BEGIN (implicit)
[sqlalchemy] CREATE TABLE Uber(charge FLOAT,pickup_time DATETIME,pickup_longitude FLOAT,dropoff_longitude FLOAT,pickup_latitude FLOAT,dropoff_latitude FLOAT,distance FLOAT);
[sqlalchemy] [generated in 0.00042s] ()
[sqlalchemy] COMMIT


In [20]:
uber.to_sql('Uber', engine, if_exists='append',index = False)

[sqlalchemy] PRAGMA main.table_info("Uber")
[sqlalchemy] [raw sql] ()
[sqlalchemy] BEGIN (implicit)
[sqlalchemy] INSERT INTO "Uber" (charge, pickup_time, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, distance) VALUES (?, ?, ?, ?, ?, ?, ?)
[sqlalchemy] [generated in 1.84986s] ((7.5, '2015-05-07 19:52:06.000000', -73.99981689453125, 40.73835372924805, -73.99951171875, 40.72321701049805, 1.6838511852242786), (7.7, '2009-07-17 20:04:56.000000', -73.994355, 40.728225, -73.99471, 40.750325, 2.458361376443877), (12.9, '2009-08-24 21:45:00.000000', -74.005043, 40.74077, -73.962565, 40.772647, 5.0379582221658445), (5.3, '2009-06-26 08:22:21.000000', -73.976124, 40.790844, -73.965316, 40.803349, 1.6622050981962735), (16.0, '2014-08-28 17:47:00.000000', -73.925023, 40.744085, -73.97308199999999, 40.761247, 4.4768549072953325), (4.9, '2011-02-12 02:27:09.000000', -73.96901899999999, 40.75591, -73.96901899999999, 40.75591, 0.0), (24.5, '2014-10-12 07:04:00.000000', -73.961

In [29]:
weather_hourly

,DATE,HourlyPrecipitation,HourlyWindSpeed
0,2009-01-01 00:00:00,0.0,18.0
1,2009-01-01 01:00:00,0.0,18.0
2,2009-01-01 02:00:00,0.0,18.0
3,2009-01-01 03:00:00,0.0,8.0
4,2009-01-01 04:00:00,0.0,11.0
...,...,...,...
77965,2015-12-31 19:00:00,0.0,6.0
77966,2015-12-31 20:00:00,0.0,10.0
77967,2015-12-31 21:00:00,0.0,0.0
77968,2015-12-31 22:00:00,0.0,7.0
